In [1]:
from scipy.linalg import expm
import scipy.sparse as sparse
from quspin.operators import hamiltonian, commutator, exp_op # Hamiltonians and operators
from quspin.basis import tensor_basis, spin_basis_1d # bases
import numpy as np # general math functions
import matplotlib.pyplot as plt # plotting library

In [2]:
L = 3 #system size
M = 0.1 #fermion mass
N=1 #number of layers
l=1 #spin length
no_checks = dict(check_pcon=False,check_symm=False,check_herm=False)
basis=spin_basis_1d(L=L,S = str(l))


In [3]:
#initial state
string = ""
for i in range(L):
    string +="0"
psi_0 = np.zeros(basis.Ns)
i_0 = basis.index(string)
psi_0[i_0] = 1.


In [4]:
def clause(a,b,k,m,n):
    zero = 0
    m = int(m)
    n = int(n)
    for l in range(L):
        if l == k:
            if (int(a[l])!=m) | (int(b[l])!=n):
                zero +=1
        else:
            if a[l]!=b[l]:
                zero +=1
    return zero

def rot(sigma,theta):
    return expm(-1j*theta*sigma/2)

def sigma_matrix(k,m,n,phi):
    sigma = np.zeros((basis.Ns,basis.Ns),dtype = complex)
    for i in range(basis.Ns):
        string_i = basis.int_to_state(basis.Ns-i-1,bracket_notation=False)
        for j in range(0,i+1):
            string_j = basis.int_to_state(basis.Ns-j-1,bracket_notation=False)
            if (clause(string_i,string_j,k,m,n)==0):
                sigma[i][j] = 0.5*(np.cos(phi)+1j*np.sin(phi))
                sigma[j][i] = 0.5*(np.cos(phi)-1j*np.sin(phi))
    sigma = sparse.csc_matrix(sigma)
    return sigma

def ms_gate(k,m,n,phi):
    return ((sigma_matrix(k,m,n,phi)+sigma_matrix(k+1,m,n,phi))**2)/2 

In [36]:
from scipy.optimize import dual_annealing
import datetime

print(datetime.datetime.now())
sigma_01_temp = [np.zeros((basis.Ns,basis.Ns),dtype = complex) for i in range(L)]
sigma_12_temp = [np.zeros((basis.Ns,basis.Ns),dtype = complex) for i in range(L)]
sigma_02_temp = [np.zeros((basis.Ns,basis.Ns),dtype = complex) for i in range(L)]

sigma_z_01_temp = [np.zeros((basis.Ns,basis.Ns),dtype = complex) for i in range(L)]
sigma_z_12_temp = [np.zeros((basis.Ns,basis.Ns),dtype = complex) for i in range(L)]
sigma_z_02_temp = [np.zeros((basis.Ns,basis.Ns),dtype = complex) for i in range(L)]


for k in range(L):
    for i in range(basis.Ns):
        for j in range(basis.Ns):

            string_i = basis.int_to_state(basis.Ns-i-1,bracket_notation=False)
            string_j = basis.int_to_state(basis.Ns-j-1,bracket_notation=False)
            
            if (clause(string_i,string_j,k,0,1)==0):
                sigma_01_temp[k][i][j] = 0.5
            if (clause(string_i,string_j,k,1,0)==0):
                sigma_01_temp[k][i][j] = 0.5
                
            if (clause(string_i,string_j,k,1,2)==0):
                sigma_12_temp[k][i][j] = 0.5
            if (clause(string_i,string_j,k,2,1)==0):
                sigma_12_temp[k][i][j] = 0.5
                
            if (clause(string_i,string_j,k,0,2)==0):
                sigma_02_temp[k][i][j] = 0.5
            if (clause(string_i,string_j,k,2,0)==0):
                sigma_02_temp[k][i][j] = 0.5
                
            if (clause(string_i,string_j,k,0,0)==0):
                sigma_z_01_temp[k][i][j] = -1.
                sigma_z_02_temp[k][i][j] = -1.
                
            if (clause(string_i,string_j,k,1,1)==0):
                sigma_z_01_temp[k][i][j] = 1.
                sigma_z_12_temp[k][i][j] = -1.

            if (clause(string_i,string_j,k,2,2)==0):
                sigma_z_02_temp[k][i][j] = 1.
                sigma_z_12_temp[k][i][j] = 1.


                
sigma_01 = np.zeros((basis.Ns,basis.Ns),dtype = complex)
sigma_12 = np.zeros((basis.Ns,basis.Ns),dtype = complex)
sigma_02 = np.zeros((basis.Ns,basis.Ns),dtype = complex)

sigma_z_01 = np.zeros((basis.Ns,basis.Ns),dtype = complex)
sigma_z_12 = np.zeros((basis.Ns,basis.Ns),dtype = complex)
sigma_z_02 = np.zeros((basis.Ns,basis.Ns),dtype = complex)

for i in range(L):
    sigma_01 +=sigma_01_temp[i]
    sigma_12 +=sigma_12_temp[i]
    sigma_02 +=sigma_02_temp[i]
    sigma_z_01 +=sigma_z_01_temp[i]
    sigma_z_12 +=sigma_z_12_temp[i]
    sigma_z_02 +=sigma_z_02_temp[i]    

    
for i in range(L):
    sigma_01_temp[i] = sparse.csc_matrix(sigma_01_temp[i])
    sigma_12_temp[i] = sparse.csc_matrix(sigma_12_temp[i])
    sigma_02_temp[i] = sparse.csc_matrix(sigma_02_temp[i])
    sigma_z_01_temp[i] = sparse.csc_matrix(sigma_z_01_temp[i])
    sigma_z_12_temp[i] = sparse.csc_matrix(sigma_z_12_temp[i])
    sigma_z_02_temp[i] = sparse.csc_matrix(sigma_z_02_temp[i])
sigma_01 = sparse.csc_matrix(sigma_01)
sigma_12 = sparse.csc_matrix(sigma_12)
sigma_02 = sparse.csc_matrix(sigma_02)

psi_var = rot(sigma_02_temp[0],10.)@rot(sigma_12_temp[0],10.)@rot(sigma_01_temp[0],10.)@psi_0
print(datetime.datetime.now())

2022-02-25 16:54:03.861959
2022-02-25 16:54:03.917049


In [40]:
print(np.allclose(sigma_01_temp[2].toarray(),sigma_matrix(2,0,1,0).toarray(),atol = 1e-8,rtol = 1e-8))

True


In [27]:
print(datetime.datetime.now())
psi_var = rot(sigma_matrix(0,0,2),10.)@rot(sigma_matrix(0,1,2),10.)@rot(sigma_matrix(0,0,1),10.)@psi_0
print(datetime.datetime.now())

2022-02-25 16:43:12.861883
2022-02-25 16:43:12.897213
